In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from mpl_toolkits.axes_grid1 import make_axes_locatable

## PDE Plot

In [ ]:
last_output = torch.load(
    "../data/hogp/pde/cpde64_seed_22.pt",
    map_location=torch.device('cpu')
)[0]

In [ ]:
best_ei = torch.tensor(last_output[0]["ei"]).argmax()
best_eihogp = torch.tensor(last_output[0]["ei_hogp_cf_smooth"]).argmax()

In [ ]:
-torch.tensor(last_output[0]["ei"]).cummax(0)[0]

In [ ]:
torch.tensor(last_output[0]["ei_hogp_cf_smooth"]).cummax(0)[0]

In [ ]:
torch.tensor(last_output[0]["ei"]).max()

In [ ]:
torch.tensor(last_output[0]["ei_hogp_cf_smooth"]).max()

In [ ]:
best_ei_par = last_output[1]["ei"][best_ei.item()]

In [ ]:
best_eihogp_par = last_output[1]["ei_hogp_cf_smooth"][best_eihogp.item()]

In [ ]:
       def cfun(tensor, k=None):
            from pde import PDE, FieldCollection, ScalarField, UnitGrid

            a = tensor[0].item()
            b = tensor[1].item()
            d0 = tensor[2].item()
            d1 = tensor[3].item()

            eq = PDE(
                 {
                    "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
                    "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
                 }
            )

            # initialize state
            grid = UnitGrid([64, 64])
            u = ScalarField(grid, a, label="Field $u$")
            v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$", seed=10)
            state = FieldCollection([u, v])

            sol = eq.solve(state, t_range=20, dt=1e-3)
            sol_tensor = torch.stack(
               (torch.from_numpy(sol[0].data), torch.from_numpy(sol[1].data))
            )
            sol_tensor[~torch.isfinite(sol_tensor)] = 1e5 * torch.randn_like(sol_tensor[~torch.isfinite(sol_tensor)])
            return sol_tensor


In [ ]:
best_ei_obj = cfun(best_ei_par)
best_eihogp_obj = cfun(best_eihogp_par)

In [ ]:
def make_pde_plot(arr, loc, use_bounds=False):
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    # f = ax[0].imshow((arr[0] - arr[0].mean()) / arr[0].std())
    vmin = None
    vmax = None
    if use_bounds:
        vmin = arr[0].mean() - 2e-12#3. * arr[0].std()
        vmax = arr[0].mean() + 2e-12#3. * arr[0].std()
    f = ax[0].imshow(arr[0] - arr[0].mean(), vmin=vmin, vmax=vmax)

    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[0])

    vmin = None
    vmax = None
    if use_bounds:
        vmin = arr[1].mean() - 2e-13 #3. * arr[1].std()
        vmax = arr[1].mean() + 2e-13#3. * arr[1].std()
    # f = ax[1].imshow((arr[1] - arr[1].mean()) / arr[1].std())
    f = ax[1].imshow(arr[1] - arr[1].mean(), vmin=vmin, vmax=vmax)

    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[1])

    ax[0].set_title("Field u", fontsize = 20)
    ax[1].set_title("Field v", fontsize = 20)
    
    plt.savefig(loc, bbox_inches="tight")

In [ ]:
make_pde_plot(best_ei_obj, "./hogp_pde_ei.pdf", use_bounds=False)

In [ ]:
make_pde_plot(best_eihogp_obj, "./hogp_pde_eihogp.pdf", use_bounds=False)

In [ ]:
rand_sol = cfun(torch.tensor([1., 3., 1., 0.1]))

In [ ]:
make_pde_plot(rand_sol, "./hogp_pde_random.pdf")

In [ ]:
best_eihogp_obj[0].mean()

## HOGP Plot

In [ ]:
last_output = torch.load(
    "../data/hogp/rfc/rf_longest//rfc_seed_1.pt",
    map_location=torch.device('cpu')
)[0]

In [ ]:
torch.tensor(last_output[0]["ei"]).max()

In [ ]:
best_ei = torch.tensor(last_output[0]["ei"]).argmax()
best_eihogp = torch.tensor(last_output[0]["ei_hogp_cf"]).argmax()

In [ ]:
best_ei_par = last_output[1]["ei"][best_ei.item()]

In [ ]:
best_eihogp_par = last_output[1]["ei_hogp_cf"][best_eihogp.item()]

In [ ]:
import sys
sys.path.append("../../fbc-maveric-research/")
from fbc.maveric.simulation_data.simulated_rsrp import SimulatedRSRP

min_Tx_power_dBm, max_Tx_power_dBm = 30, 50

# loads the 11 powermap files that are typically mounted in
# "/mnt/shared/yuchenq/power_maps/*.npz"
# and does the same type of powermap construction as in
# fbc.maveric.simulation_data.simulated_rsrp.construct_from_npz_files

import glob
import json
powermaps = glob.glob("../data/powermatrixDT*.json")
all_pmap_dicts = []
for pmap_loc in powermaps:
    with open(pmap_loc, "r") as f:
        all_pmap_dicts.append(json.load(f))

downtilts_maps = {}
for i in range(11):
    downtilts_maps[float(i)] = SimulatedRSRP.build_single_powermap(
        all_pmap_dicts[i]
    )

simulated_rsrp = SimulatedRSRP(
    downtilts_maps=downtilts_maps,
    min_TX_power_dBm=min_Tx_power_dBm,
    max_TX_power_dBm=max_Tx_power_dBm,
)

def simulate(theta):
    theta = theta.cpu().detach().numpy()
    downtilts = theta[:15].astype(int)
    tx_pwrs = theta[15:]
    (
        rsrp_powermap,
        interference_powermap,
        _,
    ) = simulated_rsrp.get_RSRP_and_interference_powermap((downtilts, tx_pwrs))
    # return torch.stack(
    #     (torch.tensor(rsrp_powermap), torch.tensor(interference_powermap))
    # )
    highd_res = torch.stack(
        (torch.tensor(rsrp_powermap), torch.tensor(interference_powermap))
    )
    return highd_res

In [ ]:
best_ei_img = simulate(best_ei_par)


In [ ]:
best_eihogp_img = simulate(best_eihogp_par)

In [ ]:
def make_rf_plot(arr, loc, titles=["Power", "Interference"]):
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    vmin = None
    vmax = None
    # f = ax[0].imshow((arr[0] - arr[0].mean()) / arr[0].std())
    f = ax[0].imshow(arr[0], vmin=vmin, vmax=vmax)

    divider = make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[0])

    vmin = None
    vmax = None

    f = ax[1].imshow(arr[1], vmin=vmin, vmax=vmax)

    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[1])

    ax[0].set_title(titles[0], fontsize = 20)
    ax[1].set_title(titles[1], fontsize = 20)
    
    plt.savefig(loc, bbox_inches="tight")

In [ ]:
make_rf_plot(best_ei_img, "./hogp_rf_ei.pdf")

In [ ]:
make_rf_plot(best_eihogp_img, "./hogp_rf_eihogp.pdf")

In [ ]:
        import numpy as np
    
        def construct_both_objectives(samples):
            rsrp_map = samples[..., 0, :, :]
            interference_map = samples[..., 1, :, :]

            weak_coverage_threshold = -80.0
            over_coverage_threshold = 6.0
            f_weak_coverage = torch.sigmoid(weak_coverage_threshold - rsrp_map)#.sum(
            #    dim=(-1, -2)
            #)
            size = np.prod(rsrp_map.shape[-2:])

            # over_coverage_area = (rsrp_map >= weak_coverage_threshold) & (
            #    interference_map + over_coverage_threshold > rsrp_map
            # )
            rsrp_gt_threshold = torch.sigmoid(rsrp_map - weak_coverage_threshold)
            if_gt_threshold = torch.sigmoid(
                (interference_map + over_coverage_threshold) - rsrp_map
            )
            over_coverage_area = rsrp_gt_threshold * if_gt_threshold

            over_coverage_map = (
                interference_map * over_coverage_area
                + over_coverage_threshold
                - rsrp_map * over_coverage_area
            )
            # over_coverage_map = (
            #     interference_map[over_coverage_area]
            #     + over_coverage_threshold
            #     - rsrp_map[over_coverage_area]
            # )
            g_weak_coverage = torch.sigmoid(over_coverage_map)#.sum(dim=(-1, -2))
            return f_weak_coverage.log(), g_weak_coverage.log()

In [ ]:
make_rf_plot(construct_both_objectives(best_ei_img), 
             "./hogp_rf_ei_coverage.pdf", 
             titles=["Log Over Coverage", "Log Under Coverage"]
)

In [ ]:
make_rf_plot(construct_both_objectives(best_eihogp_img), 
             "./hogp_rf_eihogp_coverage.pdf",
                         titles=["Log Over Coverage", "Log Under Coverage"]
)

## Optics Plot

In [ ]:
last_output = torch.load(
    "../data/hogp/optics/boptics_seed_20.pt",
    map_location=torch.device('cpu')
)[0]

In [ ]:
best_ei = torch.tensor(last_output[0]["ei"]).argmax()
best_eihogp = torch.tensor(last_output[0]["ei_hogp_cf"]).argmax()

In [ ]:
best_ei

In [ ]:
torch.tensor(last_output[0]["ei"]).max().exp()

In [ ]:
torch.tensor(last_output[0]["ei_hogp_cf"]).max().exp()

In [ ]:
best_ei_par = last_output[1]["ei"][best_ei.item()]

In [ ]:
best_eihogp_par = last_output[1]["ei_hogp_cf_smooth"][best_eihogp.item()]

In [ ]:
torch.tensor(last_output[0]["rnd"]).min().exp()

In [ ]:
from gym_interf import InterfEnv

def cfun(x):
    #if k not in gym_dict.keys():
    gym_dict = InterfEnv()
    # gym_dict[k].reset(actions=x[4:].cpu().detach().numpy())
    gym_dict.reset(actions=(1e-4, 1e-4, 1e-4, 1e-4))

    action = x[:4].cpu().detach().numpy()
    state = gym_dict.step(action)
    return torch.tensor(state[0]).double()

In [ ]:
worst_rand = torch.tensor(last_output[0]["rnd"]).argmin()

In [ ]:
worst_par = last_output[1]["rnd"][worst_rand.item()]

In [ ]:
worst_obj = cfun(worst_par)

In [ ]:
best_ei_obj = cfun(best_ei_par)
best_eihogp_obj = cfun(best_eihogp_par)

In [ ]:
fig, ax = plt.subplots(2, 8, figsize = (24, 6))
ax = ax.reshape(-1)
for i in range(16):
    f = ax[i].imshow(best_ei_obj[i])
    divider = make_axes_locatable(ax[i])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[i])
    
plt.subplots_adjust(wspace=0.35)
plt.savefig("./hogp_optics_ei_best.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2, 8, figsize = (24, 6))
ax = ax.reshape(-1)
for i in range(16):
    f = ax[i].imshow(best_eihogp_obj[i])
    divider = make_axes_locatable(ax[i])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[i])
    
plt.subplots_adjust(wspace=0.35)
plt.savefig("./hogp_optics_eihogp_best.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2, 8, figsize = (24, 6))
ax = ax.reshape(-1)
for i in range(16):
    f = ax[i].imshow(worst_obj[i])
    divider = make_axes_locatable(ax[i])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(f,cax=cax, ax = ax[i])
    
plt.subplots_adjust(wspace=0.35)

plt.savefig("./hogp_optics_random.pdf", bbox_inches="tight")